# 🎶 **Demo 1: Math Agent with LangChain**
In this notebook we’ll build a simple agent that can add and subtract two numbers using **LangChain** and a LLM.

### 📑 **Table of Contents**

1. 🛠️ Objectives  
2. 📦 Install Dependencies  
3. 🔑 Configure API Key  
4. 👉 Define Addition Tool (`@tool`)  
5. 🔗 Import Tools  
6. 🤖 Create LLM and Initialize Agent
7. 🎯 Interactive Tests  
8. ✅ Conclusions  


### 🛠️ **Objectives**

- Define an **addition**and a **subtraction** tool with `@tool`.  
- Create an **AgentExecutor** to orchestrate both tools.  
- Test the agent with natural prompts.  
---

In [18]:
from dotenv import load_dotenv
import os

In [19]:
# Load environment variables from .env file
load_dotenv(override=True) 

True

In [20]:
# Set the OpenAI API key from the environment variable
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

### 👉 **Define Tools with `@tool`**
We use the `@tool` decorator to register an addition function automatically.

```python
from langchain.agents import tool

@tool
def add_numbers(input_str: str) -> str:
    """
    Takes two numbers separated by a space and returns their sum.

    Args:
        input_str (str): Two numbers in the form "A B", e.g. "3 5"

    Returns:
        str: The sum A + B, or an error message if parsing fails.
    """
    try:
        a, b = map(float, input_str.split())
        return str(a + b)
    except Exception:
        return "error: please send two numbers separated by space, e.g. '3 5'"

```
#### **Key components**

1. **@tool decorator**
   - Registers the function with LangChain
   - Creates tool attributes (.name, .description, .func)
   - Generates JSON schema for validation
   - Transforms regular functions into callable tools

2. **Function name**
   - Used by LLM to select appropriate tool
   - Used as reference in chains and tool mappings
   - Appears in tool call logs for debugging
   - Should clearly indicate the tool's purpose

3. **Type annotations**
   - Enable automatic input validation
   - Create schema for parameters
   - Allow proper serialization of inputs/outputs
   - Help LLM understand required input formats

4. **Docstring**
   - Provides context for the LLM to decide when to use the tool
   - Documents parameter requirements
   - Explains expected outputs and behavior
   - Critical for tool selection by the LLM

5. **Implementation**
   - Executes the actual operation
   - Handles errors appropriately
   - Returns properly formatted results
   - Should be efficient and robust


### 🔗 **Import Tools and AgentExecutor**

In [21]:
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from src.tools.math_tools import add_numbers, subtract_numbers
from src.tools.search_tools import wikipedia_search
from langchain.agents import initialize_agent

In [22]:
print("Tools available:", add_numbers.name, subtract_numbers.name, wikipedia_search.name)

Tools available: add_numbers subtract_numbers wikipedia_search


In [23]:
print("Args:", subtract_numbers.description, 
      add_numbers.description, 
      wikipedia_search.description)

Args: Takes two numbers separated by a space and returns the result of A - B.

Args:
    input_str (str): Two numbers in the form "A B", e.g. "8 3"

Returns:
    str: The difference A − B, or an error message if parsing fails. Takes two numbers separated by a space and returns their sum.

Args:
    input_str (str): Two numbers in the form "A B", e.g. "3 5"

Returns:
    str: The sum A + B, or an error message if parsing fails. Performs a search on Wikipedia and returns a summary of the first result found.

Args:
    query (str): Search term.

Returns:
    str: Summary of the corresponding Wikipedia article.


## 🤖 **Create LLM and Initialize Agent**
Instantiate the LLM (temperature=0 for determinism) and build the agent.

In [68]:
# Configure the LLM
llm = ChatOpenAI(temperature=0, model="gpt-4o")

In [64]:
# Test the LLM with a simple question 
answer = llm.invoke("What is an agent in AI?")
print(answer.content)

In the context of artificial intelligence (AI), an "agent" refers to an entity that perceives its environment through sensors and acts upon that environment through actuators. Agents can be either software-based or physical entities, and they are designed to achieve specific goals or perform tasks.

Here are some key characteristics of agents in AI:

1. **Perception**: Agents gather information about their environment using sensors. This could involve data from cameras, microphones, or other input devices, depending on the type of agent.

2. **Action**: Based on the information they perceive, agents take actions to affect their environment. This could involve moving, making decisions, or communicating with other agents or systems.

3. **Autonomy**: Many agents operate autonomously, meaning they can make decisions and take actions without human intervention. However, some agents may require human input or oversight.

4. **Goal-oriented**: Agents are typically designed to achieve specifi

#### **Key parameters of `initialize_agent`**

1. **`tools`**

2.  **`llm`**

3. **`agent`**:
   - Specifies the reasoning framework for the agent.
   - `"zero-shot-react-description"` enables:
     - **Zero-shot reasoning**: The agent can solve tasks it hasn't seen before by thinking through the problem step by step.
     - **React framework**: A logical loop of:
       - **Reason** → Think about the task.
       - **Act** → Use a tool to perform an action.
       - **Observe** → Check the tool's output.
       - **Plan** → Decide what to do next.

4. **`verbose`**:
   - If `True`, it prints detailed logs of the agent’s thought process.
   - Useful for debugging or understanding how the agent makes decisions.

In [69]:
# Initialize the agent with the specified tools and LLM 
agent = initialize_agent(
    tools= [add_numbers, subtract_numbers, wikipedia_search],
    llm=llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

---
### 7️⃣ **Interactive Tests** 🎯

In [72]:
agent.invoke({"input": "In 2023, the US GDP was approximately $27.72 trillion, while Canada's was around $2.14 trillion and Mexico's was about $1.79 trillion what is the total.?"})



> Entering new AgentExecutor chain...
Question: What is the total GDP of the US, Canada, and Mexico in 2023?
Thought: I need to add the GDP values of the US, Canada, and Mexico to find the total.
Action:
```
{
  "action": "add_numbers",
  "action_input": "27.72 2.14"
}
```

Observation: 29.86
Thought:To find the total GDP of the US, Canada, and Mexico, I need to add the result of the previous addition (29.86) to Mexico's GDP.

Action:
```
{
  "action": "add_numbers",
  "action_input": "29.86 1.79"
}
```

Observation: 31.65
Thought:I now know the final answer.

Final Answer: The total GDP of the US, Canada, and Mexico in 2023 is approximately $31.65 trillion.

> Finished chain.


{'input': "In 2023, the US GDP was approximately $27.72 trillion, while Canada's was around $2.14 trillion and Mexico's was about $1.79 trillion what is the total.?",
 'output': 'The total GDP of the US, Canada, and Mexico in 2023 is approximately $31.65 trillion.'}

In [74]:
agent.invoke({"input": "¿Cuál es el plato típico de Antioquía y cuánto costaría invitar a 5 personas a almorzar?"})



> Entering new AgentExecutor chain...
Question: ¿Cuál es el plato típico de Antioquía y cuánto costaría invitar a 5 personas a almorzar?
Thought: Primero, buscaré información sobre el plato típico de Antioquía.
Action:
```
{
  "action": "wikipedia_search",
  "action_input": "plato típico de Antioquía"
}
```

Observation: No Wikipedia page was found for this term.
Thought:Question: ¿Cuál es el plato típico de Antioquía y cuánto costaría invitar a 5 personas a almorzar?
Thought: Dado que no encontré información en Wikipedia, intentaré proporcionar información basada en mi conocimiento previo. El plato típico de Antioquía es la bandeja paisa. Ahora, investigaré el costo promedio de una bandeja paisa para calcular el costo total para 5 personas.
Action:
```
{
  "action": "wikipedia_search",
  "action_input": "bandeja paisa"
}
```

Observation: Bandeja paisa (Spanish pronunciation: [banˌdexa ˈpai̯sa]; lit. 'Paisa platter'), with variations known as bandeja de arriero, bandeja montañera, a

{'input': '¿Cuál es el plato típico de Antioquía y cuánto costaría invitar a 5 personas a almorzar?',
 'output': 'El plato típico de Antioquía es la bandeja paisa. Invitar a 5 personas a almorzar bandeja paisa costaría aproximadamente 150,000 pesos colombianos.'}

### ✅ **Conclusions**

- We defined *add_numbers* and *subtract_numbers* tools using @tool.  
- The agent chooses and invokes the correct tool based on your prompt.  
- In just a few lines, we have a full “prompt → reasoning → tool call → response” flow.  
